# Sayou Refinery Quick Start

**Sayou Fabric**의 데이터 정제소인 `sayou-refinery`의 핵심 기능을 시연합니다.
이 라이브러리는 **Normalization(형태 변환)**과 **Processing(내용 정제)** 두 단계를 통해, 어떤 데이터든 LLM이 읽기 좋은 표준 블록(`SayouBlock`)으로 변환합니다.

**시연 시나리오:**
1. **Document Normalization:** JSON 문서를 Markdown으로 변환 + PII 마스킹
2. **HTML Normalization:** 더러운 HTML에서 태그 제거 + 텍스트 추출
3. **Record Processing:** DB 데이터의 결측치 채우기 + 이상치 제거

In [ ]:
import logging
from sayou.refinery.pipeline import RefineryPipeline

# 로그 레벨 설정 (처리 과정을 확인하기 위함)
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

print("Import Successful!")

## 1. 파이프라인 초기화 (Configuration)

Refinery를 생성하고 정제 규칙을 설정합니다.
여기서 설정한 값(`kwargs`)은 파이프라인 내의 모든 프로세서에게 전파됩니다.

* **PII Masking:** 이메일 마스킹 활성화
* **Imputation:** `category` 필드가 비어있으면 "Unknown"으로 채움
* **Outlier:** `price` 필드가 0~1000 범위를 벗어나면 1000으로 고정(clamp)

In [ ]:
# 파이프라인 생성
pipeline = RefineryPipeline()

# 전역 설정 주입 (각 Processor의 initialize 인자와 매칭됨)
pipeline.initialize(
    mask_email=True,  # PiiMasker용
    imputation_rules={"category": "Unknown"},  # Imputer용
    outlier_rules={"price": {"min": 0, "max": 1000, "action": "clamp"}}  # OutlierHandler용
)

## 2. 시나리오 1: 문서(Document) 정제

`sayou-document`가 추출했다고 가정한 JSON 데이터를 입력합니다.
* **기대 결과:**
    * 제목은 Markdown Heading(` # `)으로 변환
    * 이메일 주소는 `[EMAIL]`로 마스킹
    * 중복된 문단은 제거

In [ ]:
# 더미 문서 데이터
raw_doc = {
    "metadata": {"title": "User Report", "author": "admin@sayou.ai"},
    "pages": [
        {
            "elements": [
                {
                    "type": "text", 
                    "text": "Contact support at help@sayou.ai or call now.",
                    "raw_attributes": {"semantic_type": "heading", "heading_level": 1}
                },
                {
                    "type": "text",
                    "text": "Duplicate content paragraph.", 
                    "raw_attributes": {}
                },
                {
                    "type": "text",
                    "text": "Duplicate content paragraph.", # 중복 제거 대상
                    "raw_attributes": {}
                }
            ]
        }
    ]
}

print(">>> Running Document Normalization...")

# strategy="standard_doc" -> DocMarkdownNormalizer 선택
blocks = pipeline.run(raw_doc, strategy="standard_doc")

for b in blocks:
    print(f"[{b.type}] {b.content}")

## 3. 시나리오 2: 웹(HTML) 정제

스크립트 태그와 스타일이 섞인 더러운 HTML을 입력합니다.
* **기대 결과:** 태그가 모두 제거되고 순수한 텍스트만 추출

In [ ]:
dirty_html = """
<html>
    <style>body { color: red; }</style>
    <body>
        <h1>  Welcome  to   Sayou  Refinery </h1>
        <script>console.log('tracking user...');</script>
        <p>This is a    clean    text content.</p>
    </body>
</html>
"""

print(">>> Running HTML Normalization...")

# strategy="html" -> HtmlTextNormalizer 선택
html_blocks = pipeline.run(dirty_html, strategy="html")

for b in html_blocks:
    # repr()을 사용하여 공백 처리 확인
    print(f"[{b.type}] {repr(b.content)}")

## 4. 시나리오 3: 데이터 레코드(DB/JSON) 정제

DB에서 가져온 Row 리스트를 입력합니다.
* **기대 결과:**
    * `id=2` (결측치): `category`가 "Unknown"으로 채워짐
    * `id=3` (이상치): `price`가 99999에서 1000으로 조정됨(Clamp)

In [ ]:
db_rows = [
    {"id": 1, "item": "Apple", "price": 500, "category": "Fruit"},
    {"id": 2, "item": "Banana", "price": 1500, "category": None}, # 결측치 발생
    {"id": 3, "item": "Diamond", "price": 99999, "category": "Gem"} # 이상치 발생
]

print(">>> Running Record Normalization...")

# strategy="json" -> RecordNormalizer 선택
record_blocks = pipeline.run(db_rows, strategy="json")

for b in record_blocks:
    print(f"[{b.type}] {b.content}")